# Design

There's plenty of lorawan chips on the market, take a look at the semtech-website to check them out. All other lora-modules out there are integrated versions of one of these semtech modules (e.g.: LLCC68 -> DL-LLCC68, sx127x, sx126x, 

# LLCC68
This forms the basis for plenty of modules, such as the DL-LLC68_s module: \
<img src="../images/wireless/lora_DL-LLCC68-S-868S.png" width=600 />

Example schematic:lora_semtech_llc68_ex.png
<img src="../images/wireless/lora_semtech_llc68_ex.png" width=700 />

## Block Diagram

Components:
1. Front-end
2. Digital modem, with modulation options: LoRa (BW: 125, 250, 500 kHz), (G)FSK
3. Digital interface and control through SPI
4. Power distribution (DC-DC and linear regulator), depending on efficiency and other req.
<img src="../images/wireless/lora_semtech_llc68_bd.png" width=700 />

## Crystal
### External oscillator requirements
See table 3-4: crystal specifications
- Frequency: 32 MHz
- Load capacitance: 10 pF
- Shunt capacitance: 0.3-2 pF (typical 0.6)
- Rseries: 30 ohms (typical)
- Driver level: 100 uW (typical)
- Crystal motion capcitance: 1.3 - 2.5 fF
- Max trimming compensation: 15-30 ppm

### Internal oscillators
- 64 kHz (Used for wake-up, and other commands referenced as RTC)
- 13 MHz (SPI communication without activating the 32 MHz ext. clock)

#### Design

**2 Options:**
- TCXO (Temperature compensated crystal oscillator) supplied by internal regulator [extreme cases, check 4.1.4]
- OSC (on-chip crystal oscillator using an external crystal resonator)

**Solving drift:**
- total frequency drift should be minimized [lower than freq_drift_max = BW_L / (3*2**SF)]

**Control block:**
External foot capacitors are not required for the 32 MHz clock.
- There are capacitances on XTA and XTB settable between: 11.3 pF - 33.4 pF
- The steps are 0.47 pF

In [14]:
### Crystal resonator choice: X322532MOB4SI (18 was the other one
# Datasheet: https://www.micros.com.pl/mediaserver/Q32.000_3.2x2.5_0006.pdf
R_SHUNT = 40 #  Ohms
C_L = 12e-12 # Load capacitance in F
C_shunt = 7e-12 # Shunt capacitance in F
f_stability = 20 # PPM
f_temp = 10 # PPM
f_aging = 3 # PPM per year
freq = 32e6 # Frequency in Hz
drive_level = 100e-6 # Max in Watts

# MCU pin capacitance
## Stray capacitanc
C_S = 5e-12 # Stray capacitance (pin and board capacitance), assumption

In [15]:
### Calcualting necessary load capacitance
### Example calculation: ABLS-8.000MHZ-20-B-3-H-T
# Assume C_L1 = C_L2
C_L1 = (C_L - C_S)*2
C_L2 = C_L1
print(f"Load capacitances: Cl1 {C_L1*1e12}pF, Cl2 {C_L2*1e12}pF")
# Load capacitances can simply be set between 11.3-33.4 pF, so OK

Load capacitances: Cl1 14.000000000000002pF, Cl2 14.000000000000002pF


In [4]:
15 / 2 - 18 # So load capacitance is seena s being 10 pF??

-10.5

**Source:**
> https://ecsxtal.com/124-quartz-crystal-design-parameters/#:~:text=The%20motional%20capacitance%20(C1)%2C,as%20illustrated%20in%20Figure%20D.


## Power
**Choice between:**
- a DC-DC buck converter (optional)
- an LDO (always present)
**Supply Current:**
- RX-mode: max 10.1 mA
- TX-mode: max 118 mA (22 dBm)
**Supply pins:**
- One can connect both
- At any time VBAT_IO <= VBAT (MUST BE!!!)


### Supply Voltage
- VBAT_IO = 3.3 V (Supply for digital I/O interface pin, except DIO3)
- VBAT = 3.3 V (Supply for RFIC)
#### DC-DC Supply
- STDBY_XOSC: frequency: 3.25 MHz OR 5 MHz
- Power consumption is reduced (current consumption almost halved!)
- 47 nH before pin 1, 15 uH inductor between pins 7 and 9
- Current use: max 120 mA (so take 150 mA) through inductor.
  
#### LDO
- Higher power consumption
- No 15 uH inductor is required
- Maintains DC-DC converter stability

## Impedance / Matching
- Transmit output power: defined into 50 ohm load impedance)
- Internal RF circuits matched

## Receive-Transmit specifications
**Bandwidth:**
- 125, 250 and 500 kHz can be the BW's depending on the spreading factor.

### Receive
- LoRa-sensitivity -129 dBm ... -117 dBm


## Transmit
- RF max output power: 22dBm
- RF-output range: TX_MAX_OUT - 31 ... TX_MAX_OUT (so 22 dBm)

### Power supply
Power is supplied by the regulaor VR_PA, connection between VR_PA and RFO is done externally to the chip.
- VR_PA and RFO are connected externally.
- VR_PA is supplied through VDD_IN (taken from the battery). The max voltage is limited by this supply.

**Source:**
> https://circuitcellar.com/research-design-hub/basics-of-design/blocking-and-interference-rejection/#:~:text=Co%2Dchannel%20rejection%20measures%20the,channel%20as%20the%20intended%20signal.

## Digital communication
### SPI (8.2.1)
- CPOL=0, CPHA=0
- Slave-only
- Speed up to 16 MHz

### Reset
Toggling this chip will lead to 
1. a complete factory reset.
2. followed by a standard calibration procedure.

### DIO
#### BUSY control line
- Internal state machine status:
    - LOW when idle, radio ready to accept commands
- T_SW: time for the BUSY-edge to rise (600 nS max)

#### DIO lines
**DIO1:**
- Generic IRQ line

**DIO2:**
- Can be configured to drive RF switch
- Generic IRQ line

**DIO3:**
- Generic IRQ line
- Automatically control TCXO (Temperature compensated oscillator, 4.1.4)

<img src="../images/wireless/lora_semtech_llc68_dio.png" width=700 />

#### Possible interrupt sources:
- TxDone, RxDone, PreambleDetected
- HeaderValid, HeaderErr, CrcErr
- Channel activity detection done, channel activity detected
- Timeout

**Sources:**\
ElektorMag - LoRaWan options: 
> <a href = "../documents/elektormag_EN2023011.pdf"> elektormag: on lorawan 

Github examples:
> https://github.com/tudsat-rocket/frodom_fancyfins

Heat dissipation application note:
> https://cdn.sparkfun.com/assets/f/f/b/4/2/SX1262_AN-Recommendations_for_Best_Performance.pdf